In [50]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from plotly import tools
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import gc

import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('BitcoinHistoricalData.csv')
print("Data shape:", df.shape)
df.head()

Data shape: (3669, 7)


,Date,Price,Open,High,Low,Vol.,Change %
0,"Aug 02, 2020","11,105.8","11,802.6","12,061.1","10,730.7",698.62K,-5.91%
1,"Aug 01, 2020","11,803.1","11,333.2","11,847.7","11,226.1",611.47K,4.14%
2,"Jul 31, 2020","11,333.4","11,096.5","11,434.8","10,964.6",530.95K,2.14%
3,"Jul 30, 2020","11,096.2","11,105.8","11,164.4","10,861.6",501.14K,-0.09%
4,"Jul 29, 2020","11,105.9","10,908.4","11,336.5","10,771.8",576.83K,1.81%


In [2]:
import pandas as pd
import datetime
import pytz

# Định nghĩa hàm chuyển đổi cho các timestamp trong file csv
def dateparse(time_in_secs):
    # Chuyển đổi chuỗi ngày tháng thành datetime
    return pd.to_datetime(time_in_secs, format='%b %d, %Y')

# Đọc dữ liệu từ file CSV
data = pd.read_csv('BitcoinHistoricalData.csv', parse_dates=[0], date_parser=dateparse)

C:\Users\Admin\AppData\Local\Temp\ipykernel_38328\2855237414.py:11: FutureWarning:

The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.



In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3669 entries, 0 to 3668
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      3669 non-null   datetime64[ns]
 1   Price     3669 non-null   object        
 2   Open      3669 non-null   object        
 3   High      3669 non-null   object        
 4   Low       3669 non-null   object        
 5   Vol.      3669 non-null   object        
 6   Change %  3669 non-null   object        
dtypes: datetime64[ns](1), object(6)
memory usage: 200.8+ KB


In [58]:
print(data.columns)

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %'], dtype='object')


In [4]:
# Đầu tiên, sửa các cột để phù hợp với tên cột hiện tại
data['Vol.'].fillna(value=0, inplace=True)  # Thay 'Volume_(BTC)' bằng 'Vol.'
data['Price'].fillna(value=0, inplace=True)  # Không cần thay đổi vì 'Price' đã đúng

# Tiếp theo, sửa các cột OHLC
data['Open'].fillna(method='ffill', inplace=True)
data['High'].fillna(method='ffill', inplace=True)
data['Low'].fillna(method='ffill', inplace=True)
data['Price'].fillna(method='ffill', inplace=True)  # Sử dụng 'Price' thay vì 'Close'

# Hiển thị dữ liệu
print(data.head())

        Date     Price      Open      High       Low     Vol. Change %
0 2020-08-02  11,105.8  11,802.6  12,061.1  10,730.7  698.62K   -5.91%
1 2020-08-01  11,803.1  11,333.2  11,847.7  11,226.1  611.47K    4.14%
2 2020-07-31  11,333.4  11,096.5  11,434.8  10,964.6  530.95K    2.14%
3 2020-07-30  11,096.2  11,105.8  11,164.4  10,861.6  501.14K   -0.09%
4 2020-07-29  11,105.9  10,908.4  11,336.5  10,771.8  576.83K    1.81%


C:\Users\Admin\AppData\Local\Temp\ipykernel_38328\3669318625.py:2: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\Admin\AppData\Local\Temp\ipykernel_38328\3669318625.py:3: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

In [5]:
import pandas as pd
import datetime
import pytz

# Giả sử bạn đã đọc dữ liệu vào DataFrame 'data'
# data = pd.read_csv('BitcoinHistoricalData.csv', parse_dates=['Date'])

# Tạo khoảng thời gian hợp lệ (tz-naive)
start = datetime.datetime(2015, 1, 1, 0, 0, 0, 0)
end = datetime.datetime(2018, 11, 11, 0, 0, 0, 0)

# Chuyển đổi cột 'Date' thành tz-naive nếu cần
data['Date'] = data['Date'].dt.tz_localize(None)

# Tìm các hàng giữa thời gian bắt đầu và kết thúc và tìm hàng đầu tiên (00:00 sáng thứ Hai)
weekly_rows = data[(data['Date'] >= start) & (data['Date'] <= end)].groupby([pd.Grouper(key='Date', freq='W-MON')]).first().reset_index()

# Hiển thị kết quả
print(weekly_rows.head())

        Date  Price   Open   High    Low     Vol. Change %
0 2015-01-05  274.8  264.7  278.3  262.9   54.99K    3.82%
1 2015-01-12  269.3  264.8  273.1  263.3   25.39K    1.72%
2 2015-01-19  215.9  210.6  219.5  205.5   31.58K    2.49%
3 2015-01-26  269.2  252.1  310.9  251.5  105.71K    6.78%
4 2015-02-02  238.9  225.3  242.5  220.5   39.02K    6.02%


In [67]:
print(weekly_rows.columns)

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %'], dtype='object')


In [6]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

# Khởi tạo chế độ notebook cho Plotly
init_notebook_mode(connected=True)

# Chuyển đổi các cột thành kiểu chuỗi trước khi thay thế
weekly_rows['Open'] = weekly_rows['Open'].astype(str).str.replace(',', '').astype(float)
weekly_rows['Price'] = weekly_rows['Price'].astype(str).str.replace(',', '').astype(float)

# Tạo các trace cho biểu đồ
trace1 = go.Scatter(
    x=weekly_rows['Date'],  # Sử dụng 'Date'
    y=weekly_rows['Open'],
    mode='lines',
    name='Open'
)

trace2 = go.Scatter(
    x=weekly_rows['Date'],  # Sử dụng 'Date'
    y=weekly_rows['Price'],  # Sử dụng 'Price'
    mode='lines',
    name='Price'
)

# Thiết lập layout cho biểu đồ
layout = dict(
    title='Historical Bitcoin Prices (2015-2018) with the Slider',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label='1m', step='month', stepmode='backward'),
                dict(count=6, label='6m', step='month', stepmode='backward'),
                dict(count=12, label='1y', step='month', stepmode='backward'),
                dict(count=36, label='3y', step='month', stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(visible=True),
        type='date'
    )
)

# Tạo biểu đồ
data = [trace1, trace2]
fig = dict(data=data, layout=layout)
iplot(fig, filename="Time Series with Rangeslider")

In [71]:
   print(weekly_rows.dtypes)

Date        datetime64[ns]
Price              float64
Open               float64
High                object
Low                 object
Vol.                object
Change %            object
dtype: object


In [72]:
   print(weekly_rows[['Open', 'Price']].head())

    Open  Price
0  264.7  274.8
1  264.8  269.3
2  210.6  215.9
3  252.1  269.2
4  225.3  238.9


In [11]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

# Khởi tạo chế độ notebook cho Plotly
init_notebook_mode(connected=True)

# Hàm chuyển đổi giá trị khối lượng
def convert_volume(vol):
    if 'K' in vol:
        return float(vol.replace('K', '').replace(',', '').strip()) * 1000
    elif 'M' in vol:
        return float(vol.replace('M', '').replace(',', '').strip()) * 1_000_000
    else:
        return float(vol.replace(',', '').strip())

# Chuyển đổi cột 'Vol.' từ dạng chuỗi sang float
weekly_rows['Vol.'] = weekly_rows['Vol.'].apply(convert_volume)

# Tạo các trace cho biểu đồ
trace1 = go.Scatter(
    x=weekly_rows['Date'],  # Sử dụng 'Date'
    y=weekly_rows['Vol.'],  # Sử dụng cột đã chuyển đổi
    mode='lines',
    name='Bitcoin Volume (USD)'
)

# Thiết lập layout cho biểu đồ
layout = dict(
    title='Historical Bitcoin Volume (USD) (2015-2018) with the slider',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label='1m', step='month', stepmode='backward'),
                dict(count=6, label='6m', step='month', stepmode='backward'),
                dict(count=12, label='1y', step='month', stepmode='backward'),
                dict(count=36, label='3y', step='month', stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(visible=True),
        type='date'
    )
)

# Tạo biểu đồ
data = [trace1]
fig = dict(data=data, layout=layout)
iplot(fig, filename="Time Series with Rangeslider")

In [9]:
print(weekly_rows['Vol.'].head())

0     54.99K
1     25.39K
2     31.58K
3    105.71K
4     39.02K
Name: Vol., dtype: object


In [14]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

# Khởi tạo chế độ notebook cho Plotly
init_notebook_mode(connected=True)

# BTC Volume vs USD visualization
trace = go.Scattergl(
    y=weekly_rows['Vol.'].astype(float),  # Sử dụng 'Vol.' thay vì 'Volume_(BTC)'
    x=weekly_rows['Price'].astype(float),  # Sử dụng 'Price' thay vì 'Weighted_Price'
    mode='markers',
    marker=dict(
        color='#FFBAD2',
        line=dict(width=1)
    )
)

layout = go.Layout(
    title='BTC Volume v/s USD',
    xaxis=dict(
        title='Weighted Price',
        tickfont=dict(  # Sử dụng tickfont cho nhãn trục
            family='Courier New, monospace',
            size=14,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Volume BTC',
        tickfont=dict(  # Sử dụng tickfont cho nhãn trục
            family='Courier New, monospace',
            size=14,
            color='#7f7f7f'
        )
    )
)

# Tạo biểu đồ
data = [trace]
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='compare_webgl')

ValueError: Invalid property specified for object of type plotly.graph_objs.layout.XAxis: 'titlefont'

Did you mean "tickfont"?

    Valid properties:
        anchor
            If set to an opposite-letter axis id (e.g. `x2`, `y`),
            this axis is bound to the corresponding opposite-letter
            axis. If set to "free", this axis' position is
            determined by `position`.
        automargin
            Determines whether long tick labels automatically grow
            the figure margins.
        autorange
            Determines whether or not the range of this axis is
            computed in relation to the input data. See `rangemode`
            for more info. If `range` is provided and it has a
            value for both the lower and upper bound, `autorange`
            is set to False. Using "min" applies autorange only to
            set the minimum. Using "max" applies autorange only to
            set the maximum. Using *min reversed* applies autorange
   